# EXERCISE

Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.  `'Switzerland17'` should be `'Switzerland'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')


def energy_function():
    excel_file = pd.ExcelFile('assets/Energy Indicators.xls') # reading the exel file

    excel_file.sheet_names # getting the sheetname...

    Energy = excel_file.parse('Energy') # parsing sheetname to get a dataframe, with variable name 'Energy'...

    Energy.columns = [items.strip()for items in Energy.columns] # removing whitespaces from columns

    Energy.drop(columns = ['Unnamed: 0', 'Unnamed: 1'], inplace = True) # removing unnecessary column

    Energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'] #renaming the columns...

    # replacing missing data(..) with NaN below...
    Energy.replace(
    {'Energy Supply':
    {
        '...' : np.NaN
    },
    'Energy Supply per Capita':
    {
        '...' : np.NaN
    },
    '% Renewable':
    {
        '...' : np.NaN
    }
    }, inplace = True)

    Energy['Energy Supply'] = Energy['Energy Supply'] * 1000000 # converting energy supply to gigajoules

    Energy = Energy.iloc[17:, :] # getting rid of headers

    Energy.reset_index(drop= True, inplace = True)

    Energy = Energy.iloc[:227, :] # dropping footers

    # renaming country names, ridding numbers & parenthesis with regex as below... 
    Energy['Country'].replace(to_replace = '^Republic.+Korea$',
                  value = 'South Korea', regex = True, inplace = True)

    Energy['Country'].replace(to_replace = '[U]ni.+Ireland',
                  value = 'United Kingdom', regex = True,inplace = True )

    Energy['Country'].replace(to_replace = '[U]ni.+America',
                  value = 'United States', regex = True, inplace = True)

    Energy['Country'].replace(to_replace = '[C]hi.+Region',
                  value = 'Hong Kong', regex = True,  inplace = True)

    Energy['Country'].replace(to_replace = '\(.+\)',
                  value = '', regex = True,  inplace = True)

    Energy['Country'].replace(to_replace = '\d+',
                  value = '', regex = True,  inplace = True)

    Energy['Country'] = [country.strip() for country in Energy['Country']]  # stripping whitespaces from country names

    return Energy


def GDP_function():
    GDP = pd.read_csv('assets/world_bank.csv') #loading gdp data...
    GDP.columns = [items.strip() for items in GDP.columns] #stripping columns of whitespaces

    # renaming columns...
    GDP.rename(
        columns = {
            'Data Source' :  'Country',
            'World Development Indicators' : 'Country Code',
            'Unnamed: 2' : 'Indicator Name',    
            'Unnamed: 3' : 'Indicator Code',
            'Unnamed: 50' : 2006,
            'Unnamed: 51' : 2007,
            'Unnamed: 52' : 2008,
            'Unnamed: 53' : 2009,
            'Unnamed: 54' : 2010,
            'Unnamed: 55' : 2011,
            'Unnamed: 56' : 2012,
            'Unnamed: 57' : 2013,
            'Unnamed: 58' : 2014,
            'Unnamed: 59' : 2015,
                    },
            inplace = True
    )

    GDP.drop(index = [0,1,2,3], inplace = True) #removing headers
    GDP.reset_index(drop=True, inplace=True)

    #renaming countries...
    GDP['Country'].replace(
    {"Korea, Rep.": "South Korea", 
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"}, inplace = True)

    GDP['Country'] = [countries.strip() for countries in GDP['Country']] #stripping countries of whitespaces

    keep = ['Country', 'Country Code', 'Indicator Name', 'Indicator Code', 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
    GDP_to_use = GDP[keep]
    return GDP_to_use


def ScimEn_function():
    file = pd.ExcelFile('assets/scimagojr-3.xlsx') #loading xslx file...

    file.sheet_names #getting sheet names

    ScimEn = file.parse('Sheet1') #parsing sheetname into dataframe with variable name ScimEn

    ScimEn_to_use = ScimEn.iloc[:15, :] # getting the top fifteen

    ScimEn_to_use['Country'] = [countries.strip() for countries in ScimEn_to_use['Country']] #stripping countries of whitespaces
    
    return ScimEn_to_use


Energy, GDP, ScimEn= energy_function(), GDP_function(), ScimEn_function() #making functions a global variable


def answer_one():
    
    #merging all dataframes as below...
    merge_1 = pd.merge(
        left =ScimEn,
        right = Energy,
        on = 'Country',
        how = 'inner'
    )

    merge_2 = merge_1.merge(
        GDP, 
        on = 'Country', 
        how = 'inner'
    )

    merge_2.set_index('Country', inplace = True) # making country the index

    # columns to use...

    keep_2 = ['Rank', 'Documents', 'Citable documents', 'Citations',
    'Self-citations', 'Citations per document', 'H index',
    'Energy Supply', 'Energy Supply per Capita', '% Renewable',
    2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]

    final_df = merge_2[keep_2]
    final_df.sort_values(by = 'Rank', ascending = True, inplace=True) #sorting final dataframe
    
#     assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"

#     assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!" 
    return final_df


answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,127191000000,93,19.7549,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,90838000000,286,11.571,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,18984000000,149,10.2328,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7920000000,124,10.6005,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,30709000000,214,17.2887,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,10431000000,296,61.9454,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,13261000000,165,17.9015,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,33195000000,26,14.9691,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,10597000000,166,17.0203,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


In [2]:
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"

assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"


## Question 2
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [3]:
final_df = answer_one() #making functions a global variable


def answer_two():
    import pandas as pd
    import numpy as np

    keep_3 = [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]

    avgGDP = final_df.groupby('Country').apply(lambda x: np.nanmean(x[keep_3]))
    avgGDP = avgGDP.sort_values(ascending = False)
    return avgGDP

answer_two()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

In [4]:
assert type(answer_two()) == pd.Series, "Q2: You should return a Series!"


## Question 3
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [5]:
mean_gdp = answer_two() #making answer two function a global variable


def answer_three():
    
    mean_gdp.index[5] # gives country with the sixth largest average gdp & that's united kingdom
    
    return final_df.loc['United Kingdom', 2015] - final_df.loc['United Kingdom', 2006]

answer_three()

246702696075.3999

## Question 4
What is the mean energy supply per capita?

*This function should return a single number.*

In [6]:
def answer_four():
    
    return np.mean(final_df['Energy Supply per Capita'])

answer_four()

157.6

## Question 5
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [7]:
def answer_five():
    
    final_df['% Renewable'] = final_df['% Renewable'].astype(float)
    return (final_df['% Renewable'].idxmax(), final_df['% Renewable'].max())

answer_five()

('Brazil', 69.64803)

In [8]:
assert type(answer_five()) == tuple, "Q5: You should return a tuple!"

assert type(answer_five()[0]) == str, "Q5: The first element in your result should be the name of the country!"


## Question 6
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [9]:
def answer_six():
    # using a function to create a new column that is the ratio of Self-Citations to Total Citations
    
    def ratio(para):
        para['citation_ratio'] = para['Self-citations'] / para['Citations']
        return para

    citation_df = final_df.groupby('Country').apply(ratio)

    return (citation_df['citation_ratio'].idxmax(), np.max(citation_df['citation_ratio']))
    
answer_six()

('China', 0.6893126179389422)

In [10]:
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"

assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"


## Question 7

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

In [11]:
def answer_seven():
    # using a function to create a population column...
    
    def population_function(para):
        para['population'] = para['Energy Supply'] / para['Energy Supply per Capita']
        return para

    population_df = final_df.groupby('Country').apply(population_function)
    population_df = population_df.sort_values(by = 'population', ascending = False)
    return population_df.index[2]


answer_seven()    

'United States'

In [12]:
assert type(answer_seven()) == str, "Q7: You should return the name of the country!"


## Question 8
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [13]:
def answer_eight():
    # using a function to create a 'citable_by_person' column...

    def population(para):

        population = para['Energy Supply'] / para['Energy Supply per Capita']
        para['citable_by_person'] = para['Citable documents'] / population
        return para

    population_df = final_df.groupby('Country').apply(population)
    corr, pval = stats.pearsonr(population_df['citable_by_person'], population_df['Energy Supply per Capita'])
    return corr

answer_eight()

0.7940010435442943

In [14]:
assert answer_eight() >= -1. and answer_eight() <= 1., "Q8: A valid correlation should between -1 to 1!"


## Question 9
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [15]:
def answer_nine():

    def new_column(para):
        
        final_df['% Renewable'] = final_df['% Renewable'].astype(float)
        if para['% Renewable'] >= np.median(final_df['% Renewable']):
            return 1
        return 0
    
    final_df['HighRenew'] = final_df.apply(new_column, 1)
    final_df.sort_values(by = 'Rank', ascending = True, inplace = True)
    return final_df['HighRenew']

answer_nine()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

In [16]:
assert type(answer_nine()) == pd.Series, "Q9: You should return a Series!"


## Question 10
Use the following dictionary to group the Countries by Continent, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [17]:
def answer_ten():

    ContinentDict  = {'China':'Asia', 
                      'United States':'North America', 
                      'Japan':'Asia', 
                      'United Kingdom':'Europe', 
                      'Russian Federation':'Europe', 
                      'Canada':'North America', 
                      'Germany':'Europe', 
                      'India':'Asia',
                      'France':'Europe', 
                      'South Korea':'Asia', 
                      'Italy':'Europe', 
                      'Spain':'Europe', 
                      'Iran':'Asia',
                      'Australia':'Australia', 
                      'Brazil':'South America'}


    final_df.sort_values(by='Rank', ascending = True, inplace = True)
    final_df['Continent'] = ContinentDict.values()

    final_df['Country'] = final_df.index

    final_df.set_index('Country', inplace = True)

    final_df['population'] = final_df['Energy Supply'] / final_df['Energy Supply per Capita']

    final_df['population'] = final_df['population'].astype(float)

    return final_df.groupby('Continent')['population'].agg((np.size, np.sum, np.mean, np.std))

answer_ten()

,size,sum,mean,std
Continent,,,,
Asia,5.0,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1.0,2.331602e+07,2.331602e+07,NaN
Europe,6.0,4.579297e+08,7.632161e+07,3.464767e+07
North America,2.0,3.528552e+08,1.764276e+08,1.996696e+08
South America,1.0,2.059153e+08,2.059153e+08,NaN


In [18]:
assert type(answer_ten()) == pd.DataFrame, "Q10: You should return a DataFrame!"

assert answer_ten().shape[0] == 5, "Q10: Wrong row numbers!"

assert answer_ten().shape[1] == 4, "Q10: Wrong column numbers!"


## Question 11
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [19]:
def answer_eleven():
    
    bin1 = pd.cut(final_df['% Renewable'], 5)

    final_df['Country'] = final_df.index
    final_df.reset_index(drop = True)
    eleven = final_df.groupby(['Continent', bin1])['Country'].agg(np.size)
    eleven = eleven.dropna(how = 'any')
    return eleven

answer_eleven()

Continent      % Renewable     
Asia           (2.212, 15.753]     4.0
               (15.753, 29.227]    1.0
Australia      (2.212, 15.753]     1.0
Europe         (2.212, 15.753]     1.0
               (15.753, 29.227]    3.0
               (29.227, 42.701]    2.0
North America  (2.212, 15.753]     1.0
               (56.174, 69.648]    1.0
South America  (56.174, 69.648]    1.0
Name: Country, dtype: float64

In [20]:
assert type(answer_eleven()) == pd.Series, "Q11: You should return a Series!"

assert len(answer_eleven()) == 9, "Q11: Wrong result numbers!"


## Question 12
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a series `PopEst` whose index is the country name and whose values are the population estimate string*

In [23]:
def answer_twelve():
    final_df['population'] = final_df['population'].map('{:,}'.format)
    return final_df['population']

answer_twelve()

Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: population, dtype: object

In [24]:
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"

assert len(answer_twelve()) == 15, "Q12: Wrong result numbers!"